In [24]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
zip_path = "/content/drive/MyDrive/Colab Notebooks/dataset_for_github_resized_224x224.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import os
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD

BATCH_SIZE = 128
EPOCHS = 50


In [26]:
train_dir_1 = "/content/dataset/dataset_for_github_resized_224x224/dataset_train_test/Train"
train_dir_2 = "/content/dataset/dataset_for_github_resized_224x224/dataset_fresh_rotten/Train"
test_dir_1 = "/content/dataset/dataset_for_github_resized_224x224/dataset_train_test/Test"
test_dir_2 = "/content/dataset/dataset_for_github_resized_224x224/dataset_fresh_rotten/Test"

train_ds_1 = image_dataset_from_directory(train_dir_1, image_size=(224, 224), batch_size=BATCH_SIZE)
train_ds_2 = image_dataset_from_directory(train_dir_2, image_size=(224, 224), batch_size=BATCH_SIZE)
test_ds_1 = image_dataset_from_directory(test_dir_1, image_size=(224, 224), batch_size=BATCH_SIZE)
test_ds_2 = image_dataset_from_directory(test_dir_2, image_size=(224, 224), batch_size=BATCH_SIZE)


Found 21044 files belonging to 2 classes.
Found 21717 files belonging to 2 classes.
Found 6738 files belonging to 2 classes.
Found 5430 files belonging to 2 classes.


In [27]:
train_ds = train_ds_1.concatenate(train_ds_2)
test_ds = test_ds_1.concatenate(test_ds_2)

def normalize_images(image, label):
    image = image / 255.0
    return image, label

train_ds = train_ds.map(normalize_images)
test_ds = test_ds.map(normalize_images)


In [28]:
# CNN modeli
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

optimizer = SGD(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [29]:
# model save yolu ve early stopping ayarları
os.makedirs("saved_models", exist_ok=True)
checkpoint_path = "saved_models/cnn_model_merged.keras"

# validation loss önceki 3 epochta yeteri kadar azalmadıysa geriye gidilip overfitting olmadan önceki halinin save'i alınır
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True)
]

# model eğitimi
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)


Epoch 1/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 79s 209ms/step - accuracy: 0.5765 - loss: 0.6746 - val_accuracy: 0.6499 - val_loss: 0.6174
Epoch 2/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 60s 179ms/step - accuracy: 0.6697 - loss: 0.5977 - val_accuracy: 0.5958 - val_loss: 0.6557
Epoch 3/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 69s 205ms/step - accuracy: 0.7140 - loss: 0.5464 - val_accuracy: 0.7309 - val_loss: 0.5340
Epoch 4/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 73s 179ms/step - accuracy: 0.7581 - loss: 0.4889 - val_accuracy: 0.7765 - val_loss: 0.4760
Epoch 5/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 60s 178ms/step - accuracy: 0.7858 - loss: 0.4472 - val_accuracy: 0.7713 - val_loss: 0.4664
Epoch 6/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 90s 203ms/step - accuracy: 0.8091 - loss: 0.4102 - val_accuracy: 0.7660 - val_loss: 0.4819
Epoch 7/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 74s 181ms/step - accuracy: 0.8231 - loss: 0.3914 - val_accuracy: 0.8000 - val_loss: 0.4299
Epoch 8/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 68s 202ms/step - accuracy: 0.8276 - loss: 0